In [1]:
import datajoint as dj
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import ipyvolume.pylab as p3

In [2]:
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')

Connecting cpapadop@10.28.0.34:3306


In [3]:
fetched_mesh = (ta3p100.Mesh & ta3p100.CurrentSegmentation & 'segment_id=648518346341351503').fetch1()

In [4]:
fetched_mesh = ta3p100.Decimation35.fetch(limit=1, as_dict=True)[0]

In [97]:
class Voxel:
    def __init__(self, vertices):
        self._vertices = vertices
        
    @property
    def vertices(self):
        return self._vertices
        
    @staticmethod
    def get_bbox(vertices):
        return np.array([(np.min(axis), np.max(axis)) for axis in vertices.T])
        
    @property
    def bbox(self):
        return self.get_bbox(self.vertices)
    
    @property
    def structure(self):
        return self._structure
        
    @structure.setter
    def structure(self, structure):
        self._structure = np.zeros((), np.uint8) # I really, really, just store each voxel as an offset value vector.
        
#     @structure.indexer
#     def structure(self, index_tuple):
#         return self.structure[index_tuple] # self._structure[index_tuple]

#     def plot_voxels(self):
#         # Scale and transpose
#         center = [4, 4, 4];
#         cubesize = 2;
#         # Vertices for Line Cube. Order matters
#         X = [0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
#         Y = [0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0]
#         Z = [0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]
#         # Example two cube matrix. Unit cube and one scaled/translated cube
#         X1 = [X X*cubesize+center(1)];
#         Y1 = [Y Y*cubesize+center(2)];
#         Z1 = [Z Z*cubesize+center(3)];
#         # Single plot command for all 'cube lines'
#         plot3(X1,Y1,Z1);
        
    def cube_voxelize(self, side_length):
        bbox = self.bbox
        # Get the number of voxels to be used to create cubes (allowing for pushing past the boundaries).
        num_voxels = np.ceil((np.abs(np.subtract(*bbox.T) / side_length))).astype(int)
        
        # Create the cube voxel grid split structure. Could also sort the vertices according to the grid at this point?
        start_coord = bbox.T[0]
        cube_friendly_bbox = np.vstack((start_coord, start_coord + (num_voxels * side_length))).T
        x_edges, y_edges, z_edges = [np.arange(minimum, maximum, side_length) for minimum, maximum in cube_friendly_bbox]
        
        # This still might be slower.
        a = mesh.vertices.copy()
        a = a[a[:,2].argsort()] # First sort doesn't need to be stable.
        a = a[a[:,1].argsort(kind='mergesort')]
        a = a[a[:,0].argsort(kind='mergesort')]
        sorted_verts = a
        
        # Tag the voxels that have vertices. Keep track of the structure offset indices. But should I also keep track of voxel ids? Probably not if
        # I do it where each voxel just holds the offset.
        
        
        # Could I tag vertices for each voxel? How expensive would that be, but it would be very easy and efficient querying and manipulating afterwords.
        
        # Query number of vertices in large chunks and then split down into the medium then 1-1 voxel resolution.
        
        # Will also need to have a voxel-level "bbox" where it re-corners (moves it to the topmost-leftmost corner, or some other corner) the starting
        # voxel structure centroid (or corner coordinate, this decision affects from where the offset is calculated) to the location that agrees with the
        # existing offset distance settings and then it should be able to easily add/subtract the offset change from the offset indices stored in the voxel
        # offset vectors.

In [91]:
mesh = Voxel(fetched_mesh['vertices'])

In [92]:
bboxes = mesh.cube_voxelize(10000)

In [93]:
len(bboxes)

262

In [ ]:
p3.figure(width=width, height=height)
p3.plot_trisurf(*self.vertices.T/1000, self.triangles, **kwargs)
p3.scatter(*.T/1000, color='blue')

In [135]:
starting_voxel_count = 300
structure = np.zeros((5, 3), np.int32)
structure[0] = [3, 7, 4]
structure[1] = [5, 3, 1]

structure

array([[3, 7, 4],
       [5, 3, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int32)

In [147]:
%%time

a = mesh.vertices.copy()

a = a[a[:,2].argsort()] # First sort doesn't need to be stable.
a = a[a[:,1].argsort(kind='mergesort')]
a = a[a[:,0].argsort(kind='mergesort')]

print(a)

[[213431.765625   222495.484375    43725.24609375]
 [213457.109375   222541.9375      43622.21875   ]
 [213474.0625     222514.5625      43789.96875   ]
 ...
 [432938.5625     309118.75        72721.5       ]
 [432962.78125    308823.75        72491.90625   ]
 [432965.3125     309138.3125      72582.65625   ]]
CPU times: user 153 ms, sys: 4.02 ms, total: 157 ms
Wall time: 155 ms
